# 1. urlretrieve

이미지 ,페이지소스 저장

In [ ]:
import urllib.request as req

# 파일 URL
img_url = 'https://c2.staticflickr.com/8/7058/6895007878_6930f04b59_b.jpg'
html = 'https://google.com'

#다운받을 경로
save_path1 = 'C:/tiger.jpg'
save_path2 = 'C:/index.html'

#예외처리
try:
    file1, header1 = req.urlretrieve(img_url, save_path1)
    file2, header2 = req.urlretrieve(html, save_path2)
    
except Exception as e:
    print('Download failed')
    print(e)
else:
    #Header 정보 출력
    print(header1)
    print(header2)
    
    #다운로드 파일 정보
    print('Filename1 {}'.format(file1))
    print('Filename2 {}'.format(file2))
    
    #성공
    print('Download Succeed')
    
    

# 2. urlopen

In [ ]:
import urllib.request as req
from urllib.error import URLError,HTTPError#예외처리 중요 -> 코드의 퀄리티 완성도를 높인다.

#다운로드 경로 및 파일명
path_list = ['C:/tiger.jpg','C:/index.html']

#다운로드 리소스 url
target_url = ['https://upload.wikimedia.org/wikipedia/commons/thumb/7/73/Lion_waiting_in_Namibia.jpg/1200px-Lion_waiting_in_Namibia.jpg','http://google.com']

for i,url in enumerate(target_url):
    
    #예외처리
    try:
        #웹 수신 정보 읽기
        response = req.urlopen(url)
        
        #수신내용
        contents = response.read() #메서드 read
        
        print("-----------------------------------------------------")
        
        #상태 정보 중간 출력
        print('Header Info-{} : {}'.format(i, response.info())) #메서드 info
        print('HTTP Status Code: {}'.format(response.getcode())) #메서드 getcode
        print("-----------------------------------------------------")
        
        with open(path_list[i],'wb') as c:
            c.write(contents)
        
        
    except HTTPError as e:
        print('Download failed')
        print('HTTPError code: ', e.code)
    except URLError as e:
        print('Download failed')
        print('URL Error Reason : ', e.reason)
        
    #성공
    else:
        print()
        print('Download Succeed.')
        



# 3. 네이버 메인페이지 기사 링크 크롤링

copy selector가 만능은 아니다. 하나의 값이 아닌 전체를 가져올 때는 보통 직접 "아이디,클래스,태그,속성"를 검색해보고 접근하여 추출한다.

lxml (lxml.html.fromstring)<br>
requests 

In [ ]:
import lxml.html
import requests


def main():
    """
    네이버 메인뉴스 스탠드 스크래핑 메인함수
    """
    # 스크래핑 대상 URL
    response = requests.get("https://www.naver.com") # get방식
    
    #신문사 링크 리스트 획득
    urls = scrape_news_list_page(response)
    
    #결과출력
    for url in urls:
        #url출력
        print(url)
        #파일쓰기
        #생략
        
        
def scrape_news_list_page(response):
    #URL리스트 선언
    urls = []
    
    #태그 정보 문자열 저장
    root = lxml.html.fromstring(response.content)
    
    for  a in root.cssselect('.api_list .api_item a.api_link'):
        #링크
        url = a.get('href')
        urls.append(url)
    
    return urls
    
    
#스크래핑 시작
if __name__ == "__main__":
    main()

# 4. 완성된 스크래핑 코드 작성

* Session 이용해서 로그인 정보를 얻을 수 있다. 연결의 흐름을 유지하여 일정된 서비스를 제공할 수 있다. <br>
ex) 로그인 활성화 상태 <- Session 정보 사용
<br>

* xpath로 값 찾기 <br><br>

* 딕셔너리 형태 출력

In [ ]:
from lxml.html import fromstring, tostring
import requests


def main():
    """
    네이버 메인뉴스 스탠드 스크래핑 메인함수
    """
    #세션 사용
    session = requests.Session()
    
    # 스크래핑 대상 URL
    response = requests.get("https://www.naver.com") # get방식
    
    #신문사 링크 딕셔너리 획득
    urls = scrape_news_list_page(response)
    
    #결과출력
    for name, url in urls.items():
        
        #url출력
        print(name, url)
        
        #파일쓰기
        #생략
        
        
def scrape_news_list_page(response):
    
    #URL리스트 선언
    urls = {}
    
    #태그 정보 문자열 저장
    root = fromstring(response.content)
    
    for  a in root.xpath('//ul[@class="api_list"]/li[@class="api_item"]/a[@class="api_link"]'):
        
        #a 구조확인
#         print(a)
        
        # a의 문자열 출력
#         print(tostring(a,pretty_print=True))
    
        name, url = extract_contents(a)
        # 딕셔너리 삽입
        urls[name] =  url
        
    return urls


def extract_contents(dom):
    #링크주소
    link = dom.get('href')
    
    #신문사명
    name = dom.xpath('./img')[0].get('alt') #xpath('/img')
    return  name , link

    
#스크래핑 시작
if __name__ == "__main__":
    main()

# 5. 사이트 요청 정보 확인

* encar(엔카) 사이트 정보 수신

* Get 파라미터 요청

* 수신 데이터 디코딩(Decoding)

* 요청 URL 정보분석


In [ ]:
import urllib.request 
from urllib.parse import urlparse

# 기본 요청1(encar)
url = "http://www.encar.com"

mem = urllib.request.urlopen(url)

# 여러정보
print('type : {}'.format(type(mem)))
print('getrul: {}'.format(mem.geturl()))  #자동으로 리다이렉트된 수신 사이트주소를 리턴
print('status: {}'.format(mem.status)) # or getcode
print('headers: {}'.format(mem.getheaders())) #수신된 헤더 정보
print('getcode: {}'.format(mem.getcode())) # or tatus
print('read: {}'.format(mem.read(100).decode('utf-8'))) # 바이트수 지정 ,바이트수가 너무크면 에러가 날 수 있다.
print('parse: {}'.format(urlparse('http://www.encar.co.kr?id=test&pw=1111'))) #전체정보
print('parse: {}'.format(urlparse('http://www.encar.co.kr?id=test&pw=1111').query)) # parse를 사용하여 쿼리정보만 확인

#get방식은 주요정보가 노출이 되기때문에 공개적인 서비스(ex.신문사)에서 사용된다.
#중요하거나 원문 데이터는 post방식을 사용
# 위의 정보를 사용하여 예외처리를 할 수있기때문에 urllib를 사용







In [ ]:
# 기본 요청2(ipifi)
API = 'https://api.ipify.org'

# Get 방식 Parameter
values = {
    'format' : 'json'
#     'format' : 'jsonP'
#     'format' : 'text'
}

print('before param : {}'.format(values))
params = urllib.parse.urlencode(values)
print('after param : {}'.format(values))


# 요청 URL 생성
URL = API + "?" + params   #수신요청한 나의 아이피주소


# 수신 데이터 읽기
data = urllib.request.urlopen(URL).read()

# 수신데이터 디코딩
text = data.decode('UTF-8') # python3버젼은 기본적으로 utf-8 이기 떄문.
print('response: {}'.format(text)) # 나의 아이피가 서버에서 내려준 정보를 수신했다는 것을 나타냄



# 6. 행정안전부 사이트 RSS 데이터 수신

* RSS란?

https://www.mois.go.kr/gpms/view/jsp/rss/rss.jsp?ctxCd=1001 <br>ctxCd=1001<br>
https://www.mois.go.kr/gpms/view/jsp/rss/rss.jsp?ctxCd=1013 <br>ctxCd=1013<br>
.
.


* 반복문을 활용한 연속 요청

* 요청 URL 정보 분석

* 수신 XML 데이터 확인


In [ ]:
import urllib.request
import urllib.parse

# 행정 안전부 : https://www.mois.go.kr
# 행정 안전부 RSS API URL

API = 'https://www.mois.go.kr/gpms/view/jsp/rss/rss.jsp'

params = []

for num in [1001,1012,1013]:
    
    params.append(dict(ctxCd=num))
    
# 중간 확인
print(params)


#연속해서 4회 요청
for c in params:
    
    # 파라미터 출력
    # print(c)
    
    # URL 인코딩
    params = urllib.parse.urlencode(c)  # 딕셔너리를 url 형태로 변환  { a : b }  ->  a = b
    
    #URL완성
    url = API + '?' + params
    
    # URL 출력
    print('url :',url)
    
    # 요청
    res_data = urllib.request.urlopen(url).read()
#     print(res_data)

    # 수신 후 디코딩
    contents = res_data.decode("UTF-8")
    
    # 출력
    print("*"*100)
    print("*"*100)
    print("*"*100)
    print("*"*100)
    print(contents)

# 7. 주식 정보 가져오기

특정 사이트에서는 코드로 요청을 하는지 브라우져에서 요청을 하는지 구분을 하고 제한을 걸 수 있다.<br>
따라서 이러한 제한을 최소화하기 위해 코드로 직접 헤더정보를 입력하여 보내야 원하는 데이터를 수신할 수 있다.


* 비동기 통신

처음에 랜더링될때는 원하는 없는 정보가 제공되고 추가적으로 후에 정보를 채워넣는 통신방식

따라서 요청했을 때 원하는 정보가 없을때는 구글 개발자도구의 네트워크탭에서 다른 url을 살펴본다.

step1. preview 또는 response정보를 확인하여 원하는 정보가 있는지 본다. <br><br>
step2. request Header 에서 브라우져로 인식시키기 위한 필요 정보를 확인한다.
ex) referer:  , user-agent:<br>

step3. fake-useragent 사용 , Header 정보삽입<br>
pip install fake-useragent


In [ ]:
import json  # 제공되는 데이터 타입이 json일때 사용

import urllib.request as req
from fake_useragent import UserAgent


# Fake Header정보 (가상으로 User-agent생성)
ua = UserAgent()
# print(ua.ie)
# print(ua.msie)
# print(ua.chrome)
# print(ua.safari)
# print(ua.random)


# 헤더 정보
headers = {
    'User-agent':ua.ie,
    'referer':'https://finance.daum.net/'
}


# 다음 주식 요청 URL
url = 'https://finance.daum.net/api/search/ranks?limit=10'


# 요청
res = req.urlopen( req.Request(url, headers=headers )).read().decode('UTF-8')


#응답 데이터 확인(Json Data)
# print('res',res)



#응답 데이터 str -> json 변환 및 data 값 출력
rank_json = json.loads(res)['data']


#중간확인
# print('중간확인:',rank_json,'\n')


for elm in rank_json:
    print('순위 : {}, 금액 : {}, 회사명 : {}'.format(elm['rank'],elm['tradePrice'],elm['name']))
    #파일(csv, 엑셀 , txt) 저장 및 db 저장
    #code


